In [36]:
!pip install ydata-profiling
!pip install pingouin

In [37]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pingouin import corr
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [38]:
from google.colab import files
uploaded = files.upload()

Saving dirty_cafe_sales.csv to dirty_cafe_sales (2).csv


In [39]:
df_2023 = pd.read_csv("dirty_cafe_sales.csv")
df_2023.loc[:] # displays first and last few records of the dataframe

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


In [40]:
from ydata_profiling import ProfileReport


# Generate the profile report
profile = ProfileReport(df_2023, title="Cafe Sales Data Quality Analysis Report")

# Display the report in the notebook
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 8/8 [00:00<00:00, 10.45it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
# Identify columns with missing values
missing_values = df_2023.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Columns with missing values before handling:")
print(missing_values)

# Impute missing numerical values with the median
numerical_cols_with_missing = ['Quantity', 'Price Per Unit', 'Total Spent']
for col in numerical_cols_with_missing:
    if col in missing_values.index:
        median_val = df_2023[col].median()
        df_2023[col].fillna(median_val, inplace=True)

# Impute missing categorical values with the mode
categorical_cols_with_missing = ['Item', 'Payment Method', 'Location']
for col in categorical_cols_with_missing:
    if col in missing_values.index:
        mode_val = df_2023[col].mode()[0] # mode() can return multiple values, take the first
        df_2023[col].fillna(mode_val, inplace=True)

print("\nMissing values after handling:")
print(df_2023.isnull().sum())

Columns with missing values before handling:
Item                 333
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      2579
Location            3265
Transaction Date     460
dtype: int64

Missing values after handling:
Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64


In [51]:
# Convert 'Transaction Date' to datetime objects
df_2023['Transaction Date'] = pd.to_datetime(df_2023['Transaction Date'], errors='coerce')

# Ensure numerical columns are of numeric type
numerical_cols = ['Quantity', 'Price Per Unit', 'Total Spent']
for col in numerical_cols:
    df_2023[col] = pd.to_numeric(df_2023[col], errors='coerce')

print("Data types after correction:")
print(df_2023.dtypes)

# Identify columns with missing values after data type correction
missing_values_after_type_correction = df_2023.isnull().sum()
missing_values_after_type_correction = missing_values_after_type_correction[missing_values_after_type_correction > 0]
print("\nColumns with missing values after data type correction:")
print(missing_values_after_type_correction)

# Impute missing numerical values with the median
numerical_cols_with_missing = ['Quantity', 'Price Per Unit', 'Total Spent']
for col in numerical_cols_with_missing:
    if col in missing_values_after_type_correction.index:
        median_val = df_2023[col].median()
        df_2023[col].fillna(median_val, inplace=True)

# Impute missing categorical values with the mode
categorical_cols_with_missing = ['Item', 'Payment Method', 'Location']
for col in categorical_cols_with_missing:
    if col in missing_values_after_type_correction.index:
        mode_val = df_2023[col].mode()[0] # mode() can return multiple values, take the first
        df_2023[col].fillna(mode_val, inplace=True)

# Impute missing dates with the mode
if 'Transaction Date' in missing_values_after_type_correction.index:
    mode_date = df_2023['Transaction Date'].mode()[0]
    df_2023['Transaction Date'].fillna(mode_date, inplace=True)


print("\nMissing values after handling:")
print(df_2023.isnull().sum())

Data types after correction:
Transaction ID              object
Item                        object
Quantity                   float64
Price Per Unit             float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
dtype: object

Columns with missing values after data type correction:
Series([], dtype: int64)

Missing values after handling:
Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64


In [44]:
print("Unique values in 'Payment Method' before standardization:")
print(df_2023['Payment Method'].unique())

print("\nUnique values in 'Location' before standardization:")
print(df_2023['Location'].unique())

# Standardize 'Payment Method' column
payment_method_mapping = {
    'Credit Card': 'Credit Card',
    'Cash': 'Cash',
    'Digital Wallet': 'Digital Wallet',
    'UNKNOWN': 'UNKNOWN',
    'Debit Card': 'Debit Card', # Assuming 'Debit Card' is a valid payment method
    'Credit card': 'Credit Card' # Standardize 'Credit card' to 'Credit Card'
}
df_2023['Payment Method'] = df_2023['Payment Method'].map(payment_method_mapping)

# Standardize 'Location' column
location_mapping = {
    'Takeaway': 'Takeaway',
    'In-store': 'In-store',
    'UNKNOWN': 'UNKNOWN',
    'Drive-thru': 'Drive-thru', # Assuming 'Drive-thru' is a valid location
    'In-Store': 'In-store' # Standardize 'In-Store' to 'In-store'
}
df_2023['Location'] = df_2023['Location'].map(location_mapping)


print("\nUnique values in 'Payment Method' after standardization:")
print(df_2023['Payment Method'].unique())

print("\nUnique values in 'Location' after standardization:")
print(df_2023['Location'].unique())

Unique values in 'Payment Method' before standardization:
['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet' 'ERROR']

Unique values in 'Location' before standardization:
['Takeaway' 'In-store' 'UNKNOWN' 'ERROR']

Unique values in 'Payment Method' after standardization:
['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet' nan]

Unique values in 'Location' after standardization:
['Takeaway' 'In-store' 'UNKNOWN' nan]


In [45]:
numerical_cols = ['Quantity', 'Price Per Unit', 'Total Spent']

for col in numerical_cols:
    print(f"Analyzing outliers for '{col}':")
    Q1 = df_2023[col].quantile(0.25)
    Q3 = df_2023[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df_2023[(df_2023[col] < lower_bound) | (df_2023[col] > upper_bound)]

    print(f"  Number of outliers: {len(outliers)}")
    print(f"  Lower bound: {lower_bound}")
    print(f"  Upper bound: {upper_bound}")
    print("-" * 30)

# Decide on an outlier handling strategy (e.g., capping or removal)
# For this example, we will cap the outliers at the bounds
for col in numerical_cols:
    Q1 = df_2023[col].quantile(0.25)
    Q3 = df_2023[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_2023[col] = df_2023[col].clip(lower=lower_bound, upper=upper_bound)

print("\nOutliers handled by capping at bounds.")

Analyzing outliers for 'Quantity':
  Number of outliers: 0
  Lower bound: -1.0
  Upper bound: 7.0
------------------------------
Analyzing outliers for 'Price Per Unit':
  Number of outliers: 0
  Lower bound: -1.0
  Upper bound: 7.0
------------------------------
Analyzing outliers for 'Total Spent':
  Number of outliers: 259
  Lower bound: -8.0
  Upper bound: 24.0
------------------------------

Outliers handled by capping at bounds.


In [46]:
# Check for consistency between 'Total Spent', 'Quantity', and 'Price Per Unit'
# Allow for a small tolerance for floating-point comparisons
tolerance = 1e-6
inconsistent_rows = df_2023[
    ~np.isclose(df_2023['Total Spent'], df_2023['Quantity'] * df_2023['Price Per Unit'], atol=tolerance)
]

print("Rows with inconsistent 'Total Spent', 'Quantity', and 'Price Per Unit':")
display(inconsistent_rows)

# Decide on a strategy to handle inconsistencies (e.g., recalculate 'Total Spent')
# For this example, we will recalculate 'Total Spent' for inconsistent rows
df_2023.loc[inconsistent_rows.index, 'Total Spent'] = df_2023['Quantity'] * df_2023['Price Per Unit']

print("\nInconsistent 'Total Spent' values have been recalculated.")

# Verify consistency after recalculation
inconsistent_rows_after = df_2023[
    ~np.isclose(df_2023['Total Spent'], df_2023['Quantity'] * df_2023['Price Per Unit'], atol=tolerance)
]
print("\nRows with inconsistent 'Total Spent', 'Quantity', and 'Price Per Unit' after recalculation:")
display(inconsistent_rows_after)

Rows with inconsistent 'Total Spent', 'Quantity', and 'Price Per Unit':


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
2,TXN_4271903,Cookie,4.0,1.0,8.0,Credit Card,In-store,2023-07-19
10,TXN_2548360,Salad,5.0,5.0,24.0,Cash,Takeaway,2023-11-07
20,TXN_3522028,Smoothie,3.0,4.0,20.0,Cash,In-store,2023-04-04
25,TXN_7958992,Smoothie,3.0,4.0,8.0,UNKNOWN,UNKNOWN,2023-12-13
31,TXN_8927252,UNKNOWN,2.0,1.0,8.0,Credit Card,NaN,2023-11-06
...,...,...,...,...,...,...,...,...
9977,TXN_5548914,Juice,2.0,3.0,8.0,Digital Wallet,In-store,2023-11-04
9984,TXN_3142496,Smoothie,3.0,4.0,4.0,Cash,Takeaway,2023-07-27
9988,TXN_9594133,Cake,5.0,3.0,8.0,NaN,Takeaway,2023-02-06
9996,TXN_9659401,Juice,3.0,3.0,3.0,Digital Wallet,Takeaway,2023-06-02



Inconsistent 'Total Spent' values have been recalculated.

Rows with inconsistent 'Total Spent', 'Quantity', and 'Price Per Unit' after recalculation:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [47]:
print("First 5 rows of the cleaned DataFrame:")
display(df_2023.head())

# Optional: Generate a new profile report on the cleaned data
# from ydata_profiling import ProfileReport
# profile_cleaned = ProfileReport(df_2023, title="Cleaned Cafe Sales Data Quality Analysis Report")
# profile_cleaned.to_notebook_iframe()

First 5 rows of the cleaned DataFrame:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [50]:
from ydata_profiling import ProfileReport


# Generate the profile report
profile = ProfileReport(df_2023, title="Cafe Sales Data Quality Analysis Report")

# Display the report in the notebook
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 8/8 [00:00<00:00, 23.62it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]